# Win 3.11 ScreenSaver implementation

In this lab, you will recreate the Windows 3.11 screen saver using Python and the pygame library.

During the work you will:

1. Set up the environment for correct operation
2. Implement all the necessary functions so that the algorithm works at a basic level
3. Add some “unique feature” of your choice, which will highlight your work among others and show that you really understand the principle of work
4. Prepare the README.md file as a laboratory report and get a good grade.

As mentioned above, for your convenience, the problem is decomposed into functions. After each function there are a series of checks that will tell you whether you are working in the right direction.

Do not modify the code outside the specified locations. This may lead to instability and errors. Good luck!

P.S. If you find any mistake - please notify me


## Install all dependencies

In this section I'd like you to create a virtual environment and install all the packages. This step is optional, but I'm sure it's a good practice to use virtual environments to work with code properly. If you have never work with them, refer to [this link](https://docs.python.org/3/library/venv.html)

You can create the environment using this command

```console
python -m venv {THE_NAME_OF_YOUR_VENV_HERE}
```


To activate the virtual environment (venv) you have to run via cmd

```console
\venv\Scripts\activate
```

Then you can install all the necessary libraries from requirements.txt

```console
pip install -r requirements.txt
```


## Create the base game cycle

For your animation to work, it must be created inside a special "game loop". Now we will try to create such a loop to check that all libraries are installed correctly and you can start working on the laboratory work. Follow the instructions in the code comments below.


In [10]:
import pygame

# How many pixels will the screeen with your animation have.
# Try different sizes to check how they work
screen_width = screen_hight = 500

pygame.init()  # Initialize the game screen
screen = pygame.display.set_mode((screen_width, screen_hight))
done = None  # This variable will track if the "game" is done or not

# The game cycle
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True

    pygame.display.flip()

pygame.quit()

If you have seen a black screen, everything is fine than! We can move forward and try to complete the task

Before we move on to writing the functions of the main program loop, we need to decide on the data structures and representation of our objects, since they must be created before the game loop begins.

Our task will be as follows. Before creating a game loop, you need to ask:

1. An object (structure) in which the created stars will be stored
2. An object that will store information about each created star
3. A constant that determines the maximum number of stars
4. A constant (or not a constant, in case you want to come up with something creative) that sets the speed of the stars (that is, the change in its coordinates per animation frame)

Your task is to think about which data structures are best to choose for each task and justify your choice. Follow the comments on the code block below.


In [11]:
"""↓↓↓ YOUR CODE HERE ↓↓↓"""

star_numbers = 150  # Should it be an integer or a float number? You will be able to change the decision later
speed = 0.1  # Should it be an integer or a float number? You will be able to change the decision later
# Each star consists of an X-coordinate, a Y-coordinate, a Z-distance (distance to the star), and a color.
# What structure will you use to store the stars? What object will store information about the star? Tuple, list, dictionary? Justify your choices
stars = []
"""↑↑↑ YOUR CODE HERE ↑↑↑"""

'↑↑↑ YOUR CODE HERE ↑↑↑'

### Justification

Our needs are as follows:

- Edit the elements inside
- Append new elements during initialization
- Loop over the data every frame
- Do not care about duplicates
- Need indexing to access the elements

So, a list would fit our use the best


Great! I'm sure you did it. Now let's write a function to create a star. As we stated earlier, each star consists of an X-coordinate, a Y-coordinate, a Z-distance (distance to the star), and a color.

We will use the random module so that the new star is generated at random coordinates within some starting “field”. The Z distance will always be equal to 256 (the maximum distance of the star from us). The initial color is 0, so that the brightness of the star increases as it approaches us.

When writing a function, you need to know that the center of coordinates in pygame is in the top left corner of the window, so be sure to take this fact into account when creating the star. For convenience, our “reduced” coordinate center will be placed at the center of the screen, that is, the coordinates should have coordinates in the intervals

(- screen width // 2 : + screen width // 2, - screen height // 2, + screen height // 2)


In [12]:
import random


def new_star() -> list:
    """↓↓↓ YOUR CODE HERE ↓↓↓"""
    star = [
        random.randint(-(screen_width // 2), screen_width // 2),
        random.randint(-(screen_hight // 2), screen_hight // 2),
        256,
        0,
    ]
    """↑↑↑ YOUR CODE HERE ↑↑↑"""

    return star

Let's generate some starts and see whether they are OK or not


In [13]:
# Tests. Run for check the function

for i in range(100):
    star_sample = new_star()
    assert len(star_sample) == 4, "The star is defined by 4 numbers"
    assert (
        -(screen_width // 2) <= star_sample[0] <= screen_width // 2
    ), "coordinates should have coordinates in the intervals (- screen width: + screen width, - screen height, + screen height)"
    assert (
        -(screen_hight // 2) <= star_sample[1] <= screen_hight // 2
    ), "coordinates should have coordinates in the intervals (- screen width: + screen width, - screen height, + screen height)"
    assert star_sample[2] == 256, "Z coordinate has to be equal 256"
    assert star_sample[3] == 0, "Start color has to be equal to 0"
print("Seems fine, good job!")

Seems fine, good job!


Now let's implement the movement and verification mechanism. We need to calculate its x and y coordinates for each star at each step in accordance with the perspective (z coordinate).
We can do this as discussed in lecture using the following formulas:
$$X_s = \frac{X*256}{Z} + X_c$$
$$Y_s = \frac{X*256}{Z} + Y_c$$

$X_s$, $Y_s$ - Coordinate on screen

$X_c$, $Y_c$ - Coordinate of the center of the screen

Then we have to check if the star has gone off the screen. If this happens, we will remove this star from our list and generate a new star instead.


In [14]:
#implemented a get_coordinate method to simplify adjusting the code

def get_coordinate(star: list) -> tuple:
    return (
        (star[0] * 256 // (star[2] + 1)) + screen_width // 2,
        (star[1] * 256 // (star[2] + 1)) + screen_hight // 2,
    )

In [15]:
def move_and_check(star: list) -> list:
    """↓↓↓ YOUR CODE HERE ↓↓↓"""
    x, y = get_coordinate(star)

    star[2] -= speed  # Change Z coordinate

    # If the coordinates go beyond the screen, we generate a new star.
    if x > screen_width or y > screen_hight or x < 0 or y < 0 or 0 >= star[2]:
        star = new_star()

    # If the color has not reached maximum brightness, increase the color.
    if star[3] < 256:
        star[3] += 0.15

    #  If suddenly the color becomes more than acceptable, then set it to 255
    if  star[3] > 256 :  # Если вдруг цвет стал больше допустимого, то выставляем его как 255
        star[3] = 255

    """↑↑↑ YOUR CODE HERE ↑↑↑"""
    return star

To check that everything works as expected, I simulate a test run. If we don't get any errors during the run, then your code is written correctly (very likely)


In [16]:
stars = [new_star() for _ in range(50)]
for i in range(1000):
    for star in stars:
        move_and_check(star)
print("Seems good!")

Seems good!


We are very close to implementing the basic algorithm. Now all that is needed is to build a loop within which our functions will be called and draw the stars on the screen. Let's implement a draw_star function that will display a star on the screen. The main thing is not to forget to make the reverse transition from our selected coordinate system to the window coordinate system.


In [17]:
def draw_star(star: list) -> None:
    """↓↓↓ YOUR CODE HERE ↓↓↓"""
    x, y = get_coordinate(star)
    """↑↑↑ YOUR CODE HERE ↑↑↑"""
    pygame.draw.circle(
        screen,
        ((star[3] + x) % 256, (star[3] + x + y) % 256, (star[3] + y) % 256), #stars should be flashing with different colors
        (x, y),
        3,
    )

Let's check how your code works using a working example. Below you need to insert functions in the right places to check that your program works exactly as planned. Follow the comments in the code, we are building the entire program from scratch!


In [18]:
import pygame
import random

"""↓↓↓ YOUR CODE HERE ↓↓↓"""
screen_width = screen_hight = 500
"""↑↑↑ YOUR CODE HERE ↑↑↑"""

pygame.init()

screen = pygame.display.set_mode((screen_width, screen_hight))
done = False

"""↓↓↓ YOUR CODE HERE ↓↓↓"""
number_of_stars = 50
speed = 0.1
stars = []
"""↑↑↑ YOUR CODE HERE ↑↑↑"""

for i in range(0, number_of_stars):
    stars.append(new_star())

while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True

    screen.fill((0, 0, 0))

    for i in range(0, number_of_stars):
        s = stars[i]

        """↓↓↓ YOUR CODE HERE ↓↓↓"""
        # Move the star and check if it still appear
        s = move_and_check(s)
        """↑↑↑ YOUR CODE HERE ↑↑↑"""

        stars[i] = s

        """↓↓↓ YOUR CODE HERE ↓↓↓"""
        # draw the star on the screen
        draw_star(stars[i])
        """↑↑↑ YOUR CODE HERE ↑↑↑"""

    pygame.display.flip()
pygame.quit()

Finally! You are breathtaking (of course, if you managed to implement everything correctly. But even if you didn’t manage to implement it, don’t be upset, you will definitely succeed)!

Now you need to try to implement some cool killer feature to add some "zest" to your work. Afterwards, do not forget to fill out the RIDMI.md file and submit your work for verification in the agreed manner.
